# Data Cleaning

Now that we have cut our massive set to only 3 years or (~1 GB) of data. It is time to get it cleaned. 

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 
import numpy as np 

In [2]:
df = pd.read_csv('datasets/3years.csv', index_col='id')
df.head()

C:\Users\hunte\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (19,59,118) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\hunte\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,2015-12-01,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,2015-12-01,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.

The first thing to do is remove all joint loan types from our data. They only comprise 4% of our total loans and add a lot more complexity. 

In [3]:
df = df[df['application_type']=='Individual']

Next, we create our target feature. Any loan that currently has a status of default or was charged off will be considered a defaulted loan. Loans that are in a grace period or late will have a chance of becoming defaulted loans in the future but for now, we will not consider them as such. 

In [4]:
df['loan_status'].value_counts()

Fully Paid            640975
Current               408003
Charged Off           178251
Late (31-120 days)     12872
In Grace Period         4799
Late (16-30 days)       2428
Default                   28
Name: loan_status, dtype: int64

In [5]:
#if 'loan_status' is either of these two options, the new 'default' column will have a True value.
df['default'] = (df['loan_status'] == 'Default' ) | (df['loan_status'] == 'Charged Off')

There are several unnecessary features that I will be removing. Some have to do with joint borrowers, some have too many empty values, and others features that are unique to loans after they have defaulted. See attached documentation for full details as to why I have removed many of these. 

In [6]:
drops = ['member_id','funded_amnt','funded_amnt_inv', 'emp_title', 'loan_status','pymnt_plan','url','desc','title', 
'initial_list_status','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_pymnt_inv','total_rec_prncp',
'total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_d','last_pymnt_amnt','next_pymnt_d',
'policy_code', 'application_type','annual_inc_joint','dti_joint','verification_status_joint','delinq_amnt','revol_bal_joint',
'sec_app_fico_range_low','sec_app_fico_range_high','sec_app_earliest_cr_line','sec_app_inq_last_6mths','sec_app_mort_acc',
'sec_app_open_acc','sec_app_revol_util','sec_app_open_act_il','sec_app_num_rev_accts','sec_app_chargeoff_within_12_mths',
'sec_app_collections_12_mths_ex_med','sec_app_mths_since_last_major_derog','hardship_flag','hardship_type','hardship_reason',
'hardship_status','deferral_term','hardship_amount','hardship_start_date','hardship_end_date','payment_plan_start_date',
'hardship_length','hardship_dpd','hardship_loan_status','orig_projected_additional_accrued_interest',
'hardship_payoff_balance_amount','hardship_last_payment_amount','disbursement_method','debt_settlement_flag',
'debt_settlement_flag_date','settlement_status','settlement_date','settlement_amount','settlement_percentage',
'settlement_term']

In [7]:
#Drops any column that was in the above list from our DataFrame.
df=df.drop(columns=drops)

Now that we have significantly culled our features. Let's see what kind of datatypes that we are working with.

In [8]:
df.dtypes

loan_amnt                         float64
term                               object
int_rate                          float64
installment                       float64
grade                              object
sub_grade                          object
emp_length                         object
home_ownership                     object
annual_inc                        float64
verification_status                object
issue_d                            object
purpose                            object
zip_code                           object
addr_state                         object
dti                               float64
delinq_2yrs                       float64
earliest_cr_line                   object
fico_range_low                    float64
fico_range_high                   float64
inq_last_6mths                    float64
mths_since_last_delinq            float64
mths_since_last_record            float64
open_acc                          float64
pub_rec                           

Employment length could be an important feature and it is currently a string so we will convert it to an integer.

Writing a function to convert employment length to an integer:

In [9]:
def experience_convert(s):
    #any row that contains a less than sign in this dataframe is < 1 year which we will call 0.
    if '<' in s:
        return 0 
    #any row that's 10+ is just going to become a 10. 
    elif '10+' in s:
        return 10  
    else:
        #If any number from 0-9 is contained in the row, it will become just that integer. 
        for i in range(0,10): 
            if str(i) in s:
                return i

One issue we will encounter though is there are a lot of missing values. We will have to find what our median is and fill it in afterwards.

In [10]:
df['emp_length'].isna().sum()

77091

In [11]:
#intiating an empty list and converting each value in emp length to an integer and adding to our list.
emplist=[]
emp = df['emp_length'].dropna()
for i in emp:
    emplist.append(experience_convert(i))

In [12]:
#finding the median of our list 
np.median(emplist)

6.0

Now that we know our median, we fill in our empty values with 6 and repeat the previous step again 

In [13]:
df['emp_length'] = df['emp_length'].fillna('6')

In [14]:
#Running the same function again.
emplist=[]
emp = df['emp_length']
for i in emp:
    emplist.append(experience_convert(i))

Set our column to the list we created in the previous step.

In [15]:
df['emp_length'] = emplist

Now we have integers that we can work with.

The term of the loans is currently a string also, let's convert these to integers as well. 

In [16]:
#initiating an empty list. 
termlist = []  
for i in df['term']:
    #only need the first 2 strings in each row since every entry is either "36 months" or "60 months"
    termlist.append(int(i[:3]))

In [17]:
#setting the column equal to the list we created in the previous cell. 
df['term'] = termlist

Now let's explore what we have in terms of missing values.

In [18]:
#sum the missing values for each column
missing = df.isna().sum() 
#remove all columns that have 0 missing values
missing = missing[missing > 0] 
#converting our list to percentages as well. 
missingpercent = missing/len(df)*100 
print(missing)
print(missingpercent)

zip_code                                1
inq_last_6mths                          1
mths_since_last_delinq             604416
mths_since_last_record            1022438
revol_util                            774
last_credit_pull_d                     26
mths_since_last_major_derog        892313
open_acc_6m                        399437
open_act_il                        399436
open_il_12m                        399436
open_il_24m                        399436
mths_since_rcnt_il                 423244
total_bal_il                       399436
il_util                            513140
open_rv_12m                        399436
open_rv_24m                        399436
max_bal_bc                         399436
all_util                           399503
inq_fi                             399436
total_cu_tl                        399437
inq_last_12m                       399437
avg_cur_bal                            15
bc_open_to_buy                      13115
bc_util                           

Making a list in which there are very few missing string values so it's easier to just drop those rows from the column.

In [19]:
mislist = ['zip_code', 'last_credit_pull_d', 'mths_since_recent_bc']

In [20]:
#dropping just the rows that are missing
df = df.dropna(subset=mislist)

And another list where I will replace the missing values with the median of the list. All the columns have 5% or less missing values so this shouldn't affect our data much at all. 

In [21]:
medlist = ['inq_last_6mths', 'revol_util',  'avg_cur_bal', 'bc_open_to_buy','bc_util', 'mo_sin_old_il_acct', 
           'num_rev_accts', 'num_tl_120dpd_2m', 'percent_bc_gt_75']

I'll define a function that will go through this list and fill all the missing values with the median of that column  

In [22]:
def fixempty(col):
    med = df[col].median()
    df[col].fillna(med, inplace=True)

for i in medlist:
    fixempty(i)

Some of the columns reference the last delinquencies on certain accounts or major derogatory marks such as bankruptcies. In these instances, the higher the number in the column, the stronger the borrower. However, it appears that if the event happened long enough ago--or never happened--the column was left blank. For these features, I will be imputing -999 for the missing values. 

In [23]:
nineninelist = ['mths_since_last_delinq', 'mths_since_last_record', 'mths_since_last_major_derog','mths_since_rcnt_il', 
                'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq', 'mths_since_recent_bc']

In [24]:
df[nineninelist].agg(['min','max'])

,mths_since_last_delinq,mths_since_last_record,mths_since_last_major_derog,mths_since_rcnt_il,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,mths_since_recent_bc
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,202.0,121.0,202.0,511.0,202.0,25.0,202.0,656.0


In [25]:
#imputing -999 in these features
for i in nineninelist:
    df[i].fillna(-999, inplace=True)

Our remaining missing values. 32% is a lot of missing values and can potentially skew our data if we fill it haphazardly. We will leave these columns as they are for now.  

In [26]:
missing = df.isna().sum()
missing = (missing[missing > 0]/len(df))*100
missing

open_acc_6m     32.054382
open_act_il     32.054301
open_il_12m     32.054301
open_il_24m     32.054301
total_bal_il    32.054301
il_util         41.182387
open_rv_12m     32.054301
open_rv_24m     32.054301
max_bal_bc      32.054301
all_util        32.056083
inq_fi          32.054301
total_cu_tl     32.054382
inq_last_12m    32.054382
dtype: float64

In [27]:
#df.to_csv('datasets/cleaned.csv')

Now onto Exploratory Data Analysis. 